In [2]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')
huggingfacehub_api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")


In [3]:
from huggingface_hub import list_models

def list_most_popular(task: str):
    for rank, model in enumerate(
        list_models(filter=task, sort="downloads", direction=-1)
):
        if rank == 5:
            break
        print(f"{model.id}, {model.downloads}\n")

list_most_popular("text-classification")

C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


distilbert/distilbert-base-uncased-finetuned-sst-2-english, 9492296

cardiffnlp/twitter-roberta-base-sentiment-latest, 4935481

facebook/fasttext-language-identification, 4406897

papluca/xlm-roberta-base-language-detection, 3416151

facebook/bart-large-mnli, 3088471



In [4]:
list_most_popular("summarization")

google-t5/t5-small, 7294648

facebook/bart-large-cnn, 5152473

google-t5/t5-base, 2543101

sshleifer/distilbart-cnn-12-6, 816964

google-t5/t5-large, 487933



In [11]:
customer_portugues = """
Espero que este e-mail o encontre em meio a uma aura de compreensão, apesar da confusão de emoções que giram dentro de mim enquanto escrevo para você. Estou escrevendo para desabafar sobre a recente experiência infeliz que tive com uma de suas máquinas de café que chegou ameaçadoramente quebrada, evocando um profundo sentimento de descrença e desespero.

Para definir o cenário, deixe-me pintar um quadro do momento em que ansiosamente desembrulhei a caixa contendo minha tão esperada máquina de café. A excitação flagrante correndo em minhas veias poderia rivalizar com o fluxo vigoroso do café através de sua melhor arte de expresso. No entanto, o que descobri dentro quebrou não apenas meu espírito, mas também qualquer aparência de confiança que eu tinha depositado em sua estimada marca.

Imagine, se puder, o choque total e a descrença que tomaram conta de mim quando coloquei os olhos em uma máquina de café desgrenhada e mutilada. Seu exterior outrora elegante estava marcado pelas cicatrizes da viagem, lembrando um soldado devastado pela guerra que lutou bravamente nos campos de algum campo de batalha de café expresso. Essa demonstração de negligência de partir o coração destruiu meus sonhos de me entregar à perfeição diária do café, deixando-me emocionalmente perturbado e inconsolável
"""  

customer_email = """
I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you. I am writing to pour my heart out about the recent unfortunate experience I had with one of your coffee machines that arrived ominously broken, evoking a profound sense of disbelief and despair.

To set the scene, let me paint you a picture of the moment I anxiously unwrapped the box containing my highly anticipated coffee machine. The blatant excitement coursing through my veins could rival the vigorous flow of coffee through its finest espresso artistry. However, what I discovered within broke not only my spirit but also any semblance of confidence I had placed in your esteemed brand.

Imagine, if you can, the utter shock and disbelief that took hold of me as I laid eyes on a disheveled and mangled coffee machine. Its once elegant exterior was marred by the scars of travel, resembling a war-torn soldier who had fought valiantly on the fields of some espresso battlefield. This heartbreaking display of negligence shattered my dreams of indulging in daily coffee perfection, leaving me emotionally distraught and inconsolable
"""

from langchain import HuggingFaceHub

summarizer = HuggingFaceHub(
    repo_id="facebook/bart-large-cnn",
    model_kwargs={"temperature":0, "max_length":180},
    huggingfacehub_api_token = huggingfacehub_api_token
)
def summarize(llm, text) -> str:
    return llm(f"Summarize this: {text}!")

summarize(summarizer, customer_email)
#summarize(summarizer, customer_portugues)

'A customer\'s coffee machine arrived ominously broken, evoking a profound sense of disbelief and despair. "This heartbreaking display of negligence shattered my dreams of indulging in daily coffee perfection, leaving me emotionally distraught and inconsolable," the customer writes. "I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you," he adds.'

In [26]:
# https://github.com/huggingface/notebooks/blob/main/transformers_doc/en/quicktour.ipynb
# # https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest

from transformers import pipeline

sentiment_model = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)

In [27]:
sentiment_model("We are very happy to show you the 🤗 Transformers library.")

[{'label': 'LABEL_2', 'score': 0.9814898371696472}]

In [28]:
label_mapping = {
    "LABEL_0": "Negativo",
    "LABEL_1": "Neutro",
    "LABEL_2": "Positivo"
}

In [29]:
result = sentiment_model("We are very happy to show you the 🤗 Transformers library.")
for item in result:
    label = item['label']
    score = item['score']
    print(f"Sentimento: {label_mapping[label]}, Confiança: {score:.2f}")

Sentimento: Positivo, Confiança: 0.98


In [30]:
results = sentiment_model(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {label_mapping[result['label']]}, with score: {round(result['score'], 4)}")

label: Positivo, with score: 0.9815
label: Neutro, with score: 0.5063


In [31]:
print(sentiment_model(customer_email))

[{'label': 'LABEL_1', 'score': 0.8218598961830139}]


In [19]:
print(sentiment_model("I am so angry and sad, I want to kill myself!"))

[{'label': 'LABEL_0', 'score': 0.9788626432418823}]


In [20]:
print(sentiment_model("I am elated, I am so happy, this is the best thing that ever happened to me!"))

[{'label': 'LABEL_2', 'score': 0.9926880598068237}]


In [21]:
print(sentiment_model("I don't care. I guess it's ok, or not, I couldn't care one way or the other"))


[{'label': 'LABEL_1', 'score': 0.5958543419837952}]


In [22]:
# https://huggingface.co/docs/transformers/main/en/main_classes/pipelines
pipe = pipeline(
    task="text-classification",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)

In [23]:
pipe("This restaurant is awesome")

[{'label': 'LABEL_2', 'score': 0.9902701377868652}]

In [24]:
pipe = pipeline(model="FacebookAI/roberta-large-mnli")

C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\otimi\.cache\huggingface\hub\models--FacebookAI--roberta-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at FacebookAI/roberta-large-mnli were not used when initializing 

In [25]:
pipe("This restaurant is awesome")

[{'label': 'NEUTRAL', 'score': 0.7313140630722046}]

In [41]:
# https://huggingface.co/models?pipeline_tag=summarization&sort=trending

from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))



[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


In [35]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

template = """Given this text, decide what is the issue the customer is concerned about. Valid categories are these:
* product issues
* delivery problems
* missing or late orders
* wrong product
* cancellation request
* refund or exchange
* bad support experience
* no clear reason to be upset

Text: {email}
Category:
"""
prompt = PromptTemplate(template=template, input_variables=["email"])
llm = VertexAI(
    model="text-bison",  # Nome do modelo no Vertex AI
    temperature=0.7,
    max_output_tokens=256
)
VertexAI.model_rebuild()
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
print(llm_chain.run(customer_email))

PydanticUserError: `VertexAI` is not fully defined; you should define `_LanguageModel`, then call `VertexAI.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.9/u/class-not-fully-defined